# Data Loader

In [1]:
# NBVAL_SKIP
# This is just a note for the notebook to skip testing this cell


from importlib.metadata import version
# Imports a tool to check the version of the library installed
print("torch version:", version("torch"))
# checks your pytorch version       
print("tiktoken version:", version("tiktoken"))
# Checks you OpenAI tokenizer version (the tool that turns text into numbers)

torch version: 2.8.0
tiktoken version: 0.12.0


In [2]:
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader
# Standard imports: tiktoken for text processing, torch for tensors
# and the "Dataset/DataLoader" duo to handle the feeding of data to the AI

# In the class we define how to slice the text
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        # creates two empth lists: one for what the AI sees (Input)
        # and one for what it should predict (Target)

        token_ids = tokenizer.encode(txt, allowed_special = {"<|endoftext|>"} )
        # Converts the whole book or text into a long list of integers.

        for i in range(0, len(token_ids)-max_length, stride):
            # a loop that starts at the beginning and jumps forward by the 'stride'.
            input_chunk = token_ids[i:i+max_length]
            # Grabs a chunk of numbers of a special length (e.g., 4 words)
            target_chunk = token_ids[i+1:i+max_length+1]
            # Grabs the exact same chunk just shifter one position on the right.
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk)) # XXX try to understand what is a tensor?
            # Converts those lists of numbers into Pytorch "Tensors" (math ready lists) and saves them
    def __len__(self):
        return len(self.input_ids)
        # tells python how many total chunks we created

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
        # tells python: " When I ask for the chunk #5, give me input #5 and target #5"


# The data loader and the variables

def create_dataloader_v1(txt, batch_size, max_length, stride, shuffle = True, drop_last = True, num_workers = 0):
    tokenizer = tiktoken.get_encoding("gpt2")
    # tells the code to use the specific "vocabulary" that the original GPT 2 used.

    dataset = GPTDatasetV1(txt, tokenizer, max_length,stride)
    # Uses the class we built above to prepare all the chunks.

    dataloader = DataLoader(dataset, batch_size=batch_size,shuffle=shuffle, drop_last= drop_last, num_workers = num_workers)
    # This "Dataloader" is the delivery truck. It groups chunks into batches (eg., 8 at a time) so computer can process faster

    return dataloader

with open("requirements.txt","r",encoding="utf-8") as f:
    raw_text = f.read()

    # Opens your text file and reads it into memory as one gaint string.

vocab_size = 50257 # the total number of unique words the tokenizer knows
output_dim = 256  # each word will be represented by 256 numbers (its "meaning")
context_length = 1024 # the maximum number of words the context can look at once 

token_embeddings_layer = torch.nn.Embedding(vocab_size, output_dim)
# Creates a lookup table. When you give it a word ID, it returns 256 numbers

pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
# creates a table for positions. It gives a unique 256 - number code for "Position 1", "Position 2" etc.

 
batch_size = 8 # the number of tokens the computer should read at a go
max_length = 4 # the length of each token
dataloader = create_dataloader_v1(
    raw_text,
    batch_size=batch_size,
    max_length=max_length,
    stride=max_length
)
# This calls the helper function we wrote earlier. It takes that big raw_text, chops it into chunks of 4 (max_length),
# and jumps forward by 4 to grab the next chunk.
# The dataloader is now an "iterator" - its ready to hand us batches of data whenever we ask..


In [3]:
for batch in dataloader:
    x, y = batch

    token_embeddings = token_embeddings_layer(x)
    pos_embeddings = pos_embedding_layer(torch.arange(max_length))

    input_embeddings = token_embeddings + pos_embeddings

    break


In [4]:
print(input_embeddings.shape)

torch.Size([8, 4, 256])
